# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [598]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation. 
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [610]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [611]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Employee Scheduling and Sales Targets

**Sets:**

*  E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

*  `sales_target` = 5500 units
*  `full_time_hours_target` = 800 hours
*  `part_time_hours_target` = 320 hours
*  `full_time_overtime_target` = 100 hours
*  P1: Priority factor for sales deviation 
*  P2: Priority factor for full-time employee hours deviation
*  P3: Priority factor for part-time employee hours deviation
*  P4: Priority factor for full-time overtime deviation

**Decision Variables:**

*  `S`: Actual sales units 
*  `FH`: Actual full-time employee hours
*  `PH`: Actual part-time employee hours
*  `FO`: Actual full-time overtime hours
*  `PO`: Actual part-time overtime hours

**Objective Function:**

Minimize the total weighted deviation from targets:

```
Minimize: 
 P1 * |S - sales_target|  + 2 * P2 * |FH - full_time_hours_target| + P3 * |PH - part_time_hours_target| + P4 * |FO - full_time_overtime_target| + 3 * P4 * PO
```

**Constraints:**

*  Sales, working hours, and overtime hours must be non-negative:
    *  `S >= 0`
    *  `FH >= 0`
    *  `PH >= 0`
    *  `FO >= 0`
    *  `PO >= 0`

**Explanation:**

* **Objective Function:**  We aim to minimize the weighted sum of deviations from the various targets. 
    * Absolute values (`|...|`) are used to penalize both positive and negative deviations.
    * The priority factors (P1-P4) weight the deviations according to their importance.
    * Full-time employee hours deviation is multiplied by 2 to reflect its higher priority.
    * Part-time overtime is multiplied by 3 * P4, making its cost three times that of full-time overtime. 

* **Constraints:** Ensure that all decision variables remain non-negative, as negative values for sales, working hours, and overtime are not realistic.

This model provides a framework for optimally managing sales targets, employee hours, and overtime. You can plug in the specific priority values (P1-P4) based on your business priorities. By solving this optimization model, you can obtain the optimal values for sales, employee working hours, and overtime that minimize the overall impact of deviations from your targets. 


## 3. Generate the pyomo code

In [612]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [613]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
full_time_overtime_target = 100
P1 = 1  # Priority for sales deviation
P2 = 2  # Priority for full-time employee hours deviation
P3 = 1  # Priority for part-time employee hours deviation
P4 = 0.5  # Priority for full-time overtime deviation

# Create a model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=["Full-Time", "Part-Time"])

# Decision Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.FH = pyo.Var(within=pyo.NonNegativeReals)
model.PH = pyo.Var(within=pyo.NonNegativeReals)
model.FO = pyo.Var(within=pyo.NonNegativeReals)
model.PO = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        P1 * abs(model.S - sales_target)
        + 2 * P2 * abs(model.FH - full_time_hours_target)
        + P3 * abs(model.PH - part_time_hours_target)
        + P4 * abs(model.FO - full_time_overtime_target)
        + 3 * P4 * model.PO
    )

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.constraints = pyo.ConstraintList()
model.constraints.add(model.S >= 0)
model.constraints.add(model.FH >= 0)
model.constraints.add(model.PH >= 0)
model.constraints.add(model.FO >= 0)
model.constraints.add(model.PO >= 0)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
solver.solve(model)

# Display the results
print("Optimal Solution:")
print(f"Actual Sales (S): {pyo.value(model.S)} units")
print(f"Full-Time Hours (FH): {pyo.value(model.FH)} hours")
print(f"Part-Time Hours (PH): {pyo.value(model.PH)} hours")
print(f"Full-Time Overtime (FO): {pyo.value(model.FO)} hours")
print(f"Part-Time Overtime (PO): {pyo.value(model.PO)} hours")
```


## 4. Input problem data and try running the generated code

In [1]:
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
full_time_overtime_target = 100
P1 = 1  # Priority for sales deviation
P2 = 2  # Priority for full-time employee hours deviation
P3 = 3  # Priority for part-time employee hours deviation
P4 = 4 # Priority for full-time overtime deviation

# Create a model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=["Full-Time", "Part-Time"])

# Decision Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.FH = pyo.Var(within=pyo.NonNegativeReals)
model.PH = pyo.Var(within=pyo.NonNegativeReals)
model.FO = pyo.Var(within=pyo.NonNegativeReals)
model.PO = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        P1 * abs(model.S - sales_target)
        + 2 * P2 * abs(model.FH - full_time_hours_target)
        + P3 * abs(model.PH - part_time_hours_target)
        + P4 * abs(model.FO - full_time_overtime_target)
        + 3 * P4 * model.PO
    )

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.constraints = pyo.ConstraintList()
model.constraints.add(model.S >= 0)
model.constraints.add(model.FH >= 0)
model.constraints.add(model.PH >= 0)
model.constraints.add(model.FO >= 0)
model.constraints.add(model.PO >= 0)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Choose a suitable solver
solver.solve(model)

# Display the results
print("Optimal Solution:")
print(f"Actual Sales (S): {pyo.value(model.S)} units")
print(f"Full-Time Hours (FH): {pyo.value(model.FH)} hours")
print(f"Part-Time Hours (PH): {pyo.value(model.PH)} hours")
print(f"Full-Time Overtime (FO): {pyo.value(model.FO)} hours")
print(f"Part-Time Overtime (PO): {pyo.value(model.PO)} hours")

model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.11.1\x3a Maximum Number of Iterations
      Exceeded.
Optimal Solution:
Actual Sales (S): 5598.608550105494 units
Full-Time Hours (FH): 797.9651364903802 hours
Part-Time Hours (PH): 325.44715980091775 hours
Full-Time Overtime (FO): 99.99212679529937 hours
Part-Time Overtime (PO): 0.016666653655680628 hours


## 5. Correct the code to verify model viability (optional)

In [2]:
model.objective()

123.32097620939697

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [616]:
print(response.text)

## Mathematical Optimization Model for Employee Scheduling and Sales Targets

**Sets:**

*  E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

*  `sales_target` = 5500 units
*  `full_time_hours_target` = 800 hours
*  `part_time_hours_target` = 320 hours
*  `full_time_overtime_target` = 100 hours
*  P1: Priority factor for sales deviation 
*  P2: Priority factor for full-time employee hours deviation
*  P3: Priority factor for part-time employee hours deviation
*  P4: Priority factor for full-time overtime deviation

**Decision Variables:**

*  `S`: Actual sales units 
*  `FH`: Actual full-time employee hours
*  `PH`: Actual part-time employee hours
*  `FO`: Actual full-time overtime hours
*  `PO`: Actual part-time overtime hours

**Objective Function:**

Minimize the total weighted deviation from targets:

```
Minimize: 
 P1 * |S - sales_target|  + 2 * P2 * |FH - full_time_hours_target| + P3 * |PH - part_time_hours_target| + P4 * |FO - full_time_overtime_target| + 3 *

In [617]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (You can replace these with your actual data)
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
full_time_overtime_target = 100
P1 = 1  # Priority for sales deviation
P2 = 2  # Priority for full-time employee hours deviation
P3 = 1  # Priority for part-time employee hours deviation
P4 = 0.5  # Priority for full-time overtime deviation

# Create a model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=["Full-Time", "Part-Time"])

# Decision Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.FH = pyo.Var(within=pyo.NonNegativeReals)
model.PH = pyo.Var(within=pyo.NonNegativeReals)
model.FO = pyo.Var(within=pyo.NonNegativeReals)
model.PO = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    return (
        P1 * abs(model.S - sales_target)
        + 2 * P2 * abs(model.FH - full_time_hours_target)
        + P3 * abs(model.PH - part_time_hours_tar